<a href="https://colab.research.google.com/github/goldjunge3010/masterpraktikum/blob/main/Example_Comparing_two_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install the DLOmix package in the current environment using pip

!python -m pip install -q dlomix==0.0.4

  Preparing metadata (setup.py) ... done


In [2]:
!python -m pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00


In [3]:
# import necessary packages
import numpy as np
import pandas as pd
import tensorflow as tf
import re

import wandb
from wandb.keras import WandbCallback
from wandb.keras import WandbMetricsLogger
import wandb.apis.reports as wr

import dlomix
from dlomix import constants, data, eval, layers, models, pipelines, reports, utils
from dlomix.data import RetentionTimeDataset
from dlomix.models import PrositRetentionTimePredictor
from dlomix.models import RetentionTimePredictor
from dlomix.eval import TimeDeltaMetric




wandb: Thanks for trying out the Report API!
wandb: For a tutorial, check out https://colab.research.google.com/drive/1CzyJx1nuOS4pdkXa2XPaRQyZdmFmLmXV
wandb: 
wandb: Try out tab completion to see what's available.
wandb:   ∟ everything:    `wr.<tab>`
wandb:       ∟ panels:    `wr.panels.<tab>`
wandb:       ∟ blocks:    `wr.blocks.<tab>`
wandb:       ∟ helpers:   `wr.helpers.<tab>`
wandb:       ∟ templates: `wr.templates.<tab>`
wandb:       
wandb: For bugs/feature requests, please create an issue on github: https://github.com/wandb/wandb/issues


In [13]:
config = {
  "seq_length" : 30,
  "batch_size" : 64,
  "val_ratio" : 0.2,
  "lr" : 0.001,
  "optimizer" : "Adam",
  "loss" : "mse"
}

# Initialize WANDB
PROJECT = 'compare_two_models'
RUN = "run_12"
wandb.init(project = PROJECT, name = RUN, config = config)

In [14]:
# load small train dataset
#TRAIN_DATAPATH = 'https://raw.githubusercontent.com/goldjunge3010/masterpraktikum/main/third_pool_tresh_1_0_train.csv'
TRAIN_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix/develop/example_dataset/proteomTools_train_val.csv'

# create dataset
rtdata = RetentionTimeDataset(data_source=TRAIN_DATAPATH,
                              seq_length = config["seq_length"],
                              batch_size = config["batch_size"],
                              val_ratio = config["val_ratio"],
                              test = False,
                              sequence_col = "sequence",
                              target_col = "irt")

print(f"Batch size: {rtdata.batch_size}")
print(f"Number training samples : {len(rtdata.train_data) * rtdata.batch_size}")
print(f"Number validation samples : {len(rtdata.val_data) * rtdata.batch_size}")

Batch size: 64
Number training samples : 27136
Number validation samples : 6784


In [15]:
# create Prosit retention time predictor
model = RetentionTimePredictor(seq_length = config["seq_length"])

# create the optimizer object
optimizer = tf.keras.optimizers.Adam(learning_rate = config["lr"])

# compile the model  with the optimizer and the metrics we want to use, we can add our custom timedelta metric
model.compile(optimizer = optimizer,
              loss = config["loss"],
              metrics=['mean_absolute_error', TimeDeltaMetric()])

# train the model
history = model.fit(rtdata.train_data,
                    validation_data=rtdata.val_data,
                    epochs=15, callbacks=[WandbMetricsLogger(log_freq = "batch")] )
wandb.finish()

Epoch 1/15
424/424 [==============================] - 14s 27ms/step - loss: 705.2912 - mean_absolute_error: 18.6683 - timedelta: 19.6462 - val_loss: 225.7338 - val_mean_absolute_error: 11.1113 - val_timedelta: 10.6886
Epoch 2/15
424/424 [==============================] - 8s 20ms/step - loss: 165.5073 - mean_absolute_error: 9.4645 - timedelta: 10.0259 - val_loss: 136.3385 - val_mean_absolute_error: 8.5311 - val_timedelta: 7.6249
Epoch 3/15
424/424 [==============================] - 7s 15ms/step - loss: 123.3931 - mean_absolute_error: 8.0158 - timedelta: 8.3548 - val_loss: 109.4093 - val_mean_absolute_error: 7.5220 - val_timedelta: 7.6765
Epoch 4/15
424/424 [==============================] - 8s 18ms/step - loss: 103.5521 - mean_absolute_error: 7.2559 - timedelta: 7.5021 - val_loss: 95.8203 - val_mean_absolute_error: 6.9012 - val_timedelta: 7.2244
Epoch 5/15
424/424 [==============================] - 9s 20ms/step - loss: 92.6835 - mean_absolute_error: 6.8020 - timedelta: 6.8597 - val_loss

batch/batch_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/mean_absolute_error,█▅▄▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/timedelta,█▅▄▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch/mean_absolute_error,█▃▃▂▂▂▂▁▁▁▁▁▁▁▁
epoch/timedelta,█▃▃▂▂▂▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▄▃▂▂▂▁▁▁▁▁▁▁▁▁


In [16]:
# save the model weights
save_path = "./output/rtmodel_2"
model.save_weights(save_path)

In [27]:
class Report():
  def __init__(self, models:dict, test_set:dlomix.data.RetentionTimeDataset, project:str, title: str, description: str):
    self.entity = wandb.apis.PublicApi().default_entity
    self.project = project
    self.title = title
    self.description = description
    self.models = models
    self.test_set = test_set
    self.api = wandb.Api()

  def create_report(self, add_residuals_section = True, add_r2_section = True, add_density_section = True):
    report = wr.Report(
        project = self.project,
        title = self.title,
        description = self.description
    )

    report.blocks = [
        wr.TableOfContents()
    ]
    if add_residuals_section:
      report.blocks += self.residuals_section()
    if add_r2_section:
      report.blocks += self.r2_section()
    if add_density_section:
      report.blocks += self.density_section()
    report.save()

  def calculate_r2(self, targets, predictions):
    from sklearn.metrics import r2_score
    r2 = r2_score(targets, predictions)
    return r2

  def calculate_residuals(self, targets, predictions):
    residuals = targets - predictions
    return residuals

  def residuals_section(self):
    panel_list_models = []
    for model in self.models:
      panel_list_models.append(
        wr.CustomChart(
          query = {'summaryTable': {"tableKey" : f"results_table_{model}"}},
          chart_name='master_praktikum/hist_residuals',
          chart_fields={'value': "residuals", "name": model}
        )
      )

    residuals_block = [
        wr.H1(text = "Residuals"),
        wr.P("Lorem ipsum dolor sit amet. Aut laborum perspiciatis sit odit omnis aut aliquam voluptatibus ut rerum molestiae sed assumenda nulla ut minus illo sit sunt explicabo? Sed quia architecto est voluptatem magni sit molestiae dolores. Non animi repellendus ea enim internos et iste itaque quo labore mollitia aut omnis totam."),
        wr.PanelGrid(
          runsets=[
            wr.Runset(self.entity, self.project),
          ],
          panels = panel_list_models
        ),
        wr.HorizontalRule(),
    ]

    return residuals_block

  def r2_section(self):
      r2_block = [
          wr.H1(text = "R2"),
          wr.P("Lorem ipsum dolor sit amet. Aut laborum perspiciatis sit odit omnis aut aliquam voluptatibus ut rerum molestiae sed assumenda nulla ut minus illo sit sunt explicabo? Sed quia architecto est voluptatem magni sit molestiae dolores. Non animi repellendus ea enim internos et iste itaque quo labore mollitia aut omnis totam."),
          wr.PanelGrid(
            runsets=[
              wr.Runset(self.entity, self.project),
            ],
            panels=[
                wr.BarPlot(
                    title="R2",
                    metrics=["r2"],
                    orientation='h',
                    title_x="R2",
                    # title_y="y axis title",
                    max_runs_to_show=20,
                    max_bars_to_show=3,
                    font_size="auto",
                ),
            ]
          ),
          wr.HorizontalRule(),
      ]
      return r2_block

  def density_section(self, irt_delta95 = 5 ):
    panel_list_models = []
    targets = self.test_set.get_split_targets(split = self.test_set.main_split)
    x_min = np.min(targets)
    x_max = np.max(targets)
    for model in self.models:
      panel_list_models.append(
        wr.CustomChart(
          query = {'summaryTable': {"tableKey" : f"results_table_{model}"}},
          chart_name='master_praktikum/density_plot',
          chart_fields={'measured': "irt", "predicted": "predicted_irt", "name": model, "irt_delta95": irt_delta95}
        )
      )

    density_block = [
        wr.H1(text = "Density"),
        wr.P("Lorem ipsum dolor sit amet. Aut laborum perspiciatis sit odit omnis aut aliquam voluptatibus ut rerum molestiae sed assumenda nulla ut minus illo sit sunt explicabo? Sed quia architecto est voluptatem magni sit molestiae dolores. Non animi repellendus ea enim internos et iste itaque quo labore mollitia aut omnis totam."),
        wr.PanelGrid(
          runsets=[
            wr.Runset(self.entity, self.project),
          ],
          panels = panel_list_models
        ),
        wr.HorizontalRule(),
    ]

    return density_block

  def compare_models(self):
    for model in self.models:
      # initialize WANDB
      RUN = model
      wandb.init(project = self.project, name = RUN, config = config)

      # predict on test_test
      predictions = self.models[model].predict(self.test_set.test_data)
      predictions = predictions.ravel()
      targets = self.test_set.get_split_targets(split = self.test_set.main_split)
      # create result df
      results_df = pd.DataFrame({"sequence": self.test_set.sequences,
                                  "irt": targets,
                                  "predicted_irt": predictions,
                                  "residuals": self.calculate_residuals(targets, predictions)})
      # log df as table to wandb
      table = wandb.Table(dataframe = results_df)
      wandb.log({f"results_table_{model}": table})

      # log r2 to wandb
      r2 = self.calculate_r2(targets, predictions)
      wandb.log({"r2": r2})

      # finish run
      wandb.finish()

#TODO
# Densitiy Plot see DLOmix

In [22]:
# create predictors from saved weights
load_path = "./output/rtmodel_1"
pre_trained_model_1 = RetentionTimePredictor(seq_length=30)
pre_trained_model_1.load_weights(load_path)

load_path = "./output/rtmodel_2"
pre_trained_model_2 = RetentionTimePredictor(seq_length=30)
pre_trained_model_2.load_weights(load_path)

models = {"model_1":pre_trained_model_1, "model_2":pre_trained_model_2}

# create testset
TEST_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/main/example_datasets/RetentionTime/proteomeTools_test.csv'
test_set = RetentionTimeDataset(data_source=TEST_DATAPATH,
                              seq_length = config["seq_length"],
                              batch_size = 32,
                              test = True,
                              sequence_col = "sequence",
                              target_col = "irt")


In [27]:
df = pd.read_csv('https://raw.githubusercontent.com/goldjunge3010/masterpraktikum/main/third_pool_tresh_1_0_test.csv')
min_target = df["indexed_retention_time"].min()
print(f"min: {min_target}")
max_target = df["indexed_retention_time"].max()
print(f"max: {max_target}")

min: -9.54625219353221
max: 88.7671581087951


In [28]:
report = Report(models = models,
                test_set = test_set,
                project = PROJECT,
                title = "Comparison of models",
                description = "A quick comparison of some models")
report.compare_models()
report.create_report(add_residuals_section = True, add_r2_section = True)

187/187 [==============================] - 1s 5ms/step


r2,▁
r2,0.95032


187/187 [==============================] - 2s 8ms/step


r2,▁
r2,0.96572


In [ ]:
1